In [1]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
import numpy as np

In [2]:
df = pd.read_excel('../Data/Todo_esp/connectividad/Cobertura_banda_ancha_a_nivel_de_Entidad_Singular_de_Poblacion.xlsx', sheet_name='ESP')
df.drop(columns=['Habitantes'], inplace=True)

/home/pomilusky/anaconda3/envs/ironhack/lib/python3.8/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [3]:
# df['Comunidad Autónoma'].value_counts()

In [4]:
# Vamos a tratar por provinicas, 
df = df[df['Comunidad Autónoma']== 'Castilla y León']

In [26]:
def cambio_enti(s):
    llista = s.lower().split(' ')
    n_llista = [i.capitalize() if i != 'de' and i != 'del' else i for i in llista ]
    s = ' '.join(n_llista)
    if '-' in s:
        llista = s.lower().split('-')
        n_llista = [i.capitalize() if i != 'de' and i != 'del' else i for i in llista ]
        s = ' '.join(n_llista)
    if '(la)' in s:
        s = 'La '+s.replace('(la)','').strip()
    if '(el)' in s:
        s = 'El '+s.replace('(el)','').strip()
    if '(los)' in s:
        s = 'Los '+s.replace('(los)','').strip()
    if '(las)' in s:
        s = 'Las '+s.replace('(las)','').strip()   

    if 'de La ' in s:
        s = s.replace('de La ', 'de la ')
    if 'de Las ' in s:
        s = s.replace('de Las ', 'de las ')
    if 'de Los ' in s:
        s = s.replace('de Los ', 'de los ')
    if ' Y ' in s:
        s = s.replace(' Y ', ' y ')
    if ' O ' in s:
        s = s.replace(' O ', ' o ')
    return s
    
def process_num(num):
    return float(re.sub(r"[^\w\s.]",'',num))
#cambio_enti('Quintana de Los Prados')

In [45]:
# Ahora que tenemos todas las Entidades singulares de poblacion vamos a buscar su ubicacion en wikipedia,
def scrap_wiki_ubi(x):
    provincia, municipio, entidad = x
    n_e = cambio_enti(entidad)
    try:
        URL = f"https://es.wikipedia.org/wiki/{n_e.replace(' ','_')}"
        #print(URL)
        r = requests.get(URL)
        soup = BeautifulSoup(r.content, 'lxml') 
        bodies = soup.findAll('tbody')
        a = False
        for e in bodies:
            if 'Ubicación' in e.text:
                r_tabla = e.findAll('tr')
                a=True
                break

        if not a: 
            URL = f"https://es.wikipedia.org/wiki/{n_e.replace(' ','-')}_({municipio.replace(' ','_')})"
            r = requests.get(URL)
            soup = BeautifulSoup(r.content, 'lxml') 
            bodies = soup.findAll('tbody')
            for e in bodies:
                if 'Ubicación' in e.text:
                    r_tabla = e.findAll('tr')
                    a=True
        
        if not a: 
            URL = f"https://es.wikipedia.org/wiki/{n_e.replace(' ','_')}_({municipio.replace(' ','_')})"
            r = requests.get(URL)
            soup = BeautifulSoup(r.content, 'lxml') 
            bodies = soup.findAll('tbody')
            for e in bodies:
                if 'Ubicación' in e.text:
                    r_tabla = e.findAll('tr')
                    a=True
        if not a: 
            URL = f"https://es.wikipedia.org/wiki/{n_e.replace(' ','_')}_({provincia})"
            r = requests.get(URL)
            soup = BeautifulSoup(r.content, 'lxml') 
            bodies = soup.findAll('tbody')
            for e in bodies:
                if 'Ubicación' in e.text:
                    r_tabla = e.findAll('tr')
                    a=True
    
        for i in r_tabla:
            if 'Ubicación' in i.text:
                here = i

        lat, lon = list(here)[1].text.split('/')[1].split(',')
        
        return process_num(lat), process_num(lon)
    except:
        # print(n_e)
        # i += 1
        # if i > 1000:
        #     i = 0
        #     print('ok')
        return np.nan

In [49]:
scrap_wiki_ubi(['Girona','Berga','Berga'])

https://es.wikipedia.org/wiki/Berga


(42.1, 1.8455555555556)

In [28]:
df['coord'] = df.apply(lambda x: scrap_wiki_ubi((x['Provincia'],x['Municipio'],x['Entidad Singular de Población'])), axis=1)

In [31]:
df.coord.isna().sum()

772

In [50]:
df[df.coord.isna()].head(10)

,Comunidad Autónoma,Provincia,Municipio,Código ESP,Entidad Singular de Población,Cobertura ≥ 30Mbps,Cobertura redes fijas ≥ 100Mbps,Cobertura 3G HSPA,Cobertura 4G LTE,coord
1705,Castilla y León,Ávila,Adanero,5001000200,ESTACIÓN (LA),0.0,0.0,1.00,1.00,NaN
1711,Castilla y León,Ávila,"Aldehuela, La",5010000200,MOLINOS (LOS),0.0,0.0,1.00,1.00,NaN
1712,Castilla y León,Ávila,"Aldehuela, La",5010000300,NAVAS (LAS),0.0,0.0,0.91,0.99,NaN
1713,Castilla y León,Ávila,"Aldehuela, La",5010000400,REHOYO (EL),0.0,0.0,1.00,0.99,NaN
1714,Castilla y León,Ávila,"Aldehuela, La",5010000500,SOLANAS DEL CARRASCAL,0.0,0.0,0.97,0.99,NaN
1715,Castilla y León,Ávila,"Aldehuela, La",5010000600,SOLANILLAS (LAS),0.0,0.0,0.94,0.97,NaN
1730,Castilla y León,Ávila,Ávila,5019000400,BERNUY-SALINERO,0.0,0.0,1.00,1.00,NaN
1732,Castilla y León,Ávila,Ávila,5019000800,URRACA-MIGUEL,0.0,0.0,1.00,0.48,NaN
1733,Castilla y León,Ávila,Ávila,5019001000,BRIEVA-VICOLOZANO,1.0,1.0,1.00,1.00,NaN
1736,Castilla y León,Ávila,"Barraco, El",5022000200,CRUCERAS (LAS),0.0,0.0,1.00,1.00,NaN


In [47]:
df.head()

,Comunidad Autónoma,Provincia,Municipio,Código ESP,Entidad Singular de Población,Cobertura ≥ 30Mbps,Cobertura redes fijas ≥ 100Mbps,Cobertura 3G HSPA,Cobertura 4G LTE,coord
1704,Castilla y León,Ávila,Adanero,5001000100,ADANERO,0.88,0.0,1.0,1.0,"(40.945833333333, 4.6066666666667)"
1705,Castilla y León,Ávila,Adanero,5001000200,ESTACIÓN (LA),0.00,0.0,1.0,1.0,NaN
1706,Castilla y León,Ávila,"Adrada, La",5002000100,ADRADA (LA),1.00,0.3,1.0,1.0,"(40.299166666667, 4.635)"
1707,Castilla y León,Ávila,Albornos,5005000100,ALBORNOS,1.00,0.0,1.0,1.0,"(40.838611111111, 4.8813888888889)"
1708,Castilla y León,Ávila,Aldeanueva de Santa Cruz,5007000100,ALDEANUEVA DE SANTA CRUZ,0.52,0.0,1.0,1.0,"(40.381666666667, 5.4194444444444)"


In [36]:
df.to_csv('../Data/castilla_leon/main.csv')